In [9]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.


import numpy as np
import pandas as pd
import csss as CSSS
import sqlalchemy as sq
import matplotlib.pyplot as plt
import pytz
import datetime as dt
import pickle as pk

% matplotlib inline

In [11]:
## Close any open connections. 
import gc
for obj in gc.get_objects():
    if isinstance(obj, sq.engine.base.Engine):
        obj.dispose()
        
## Read key for PecanSt, Stored in text file
with open('../keys/pecanstkey.txt','r') as f:
    key = f.read().strip()
    f.close()
    
    
## Create engine
engine = sq.create_engine("postgresql+psycopg2://%s@dataport.pecanstreet.org:5434/postgres" % key)

## Identify homes with complete data from Pecan Street
Isolate dates in 2015 and 2016

In [12]:
query="""SELECT dataid, count(*) FROM university.electricity_egauge_15min
WHERE local_15min
BETWEEN '01-01-2015' AND '01-01-2017' GROUP BY dataid """
datacounts = pd.read_sql_query(query,engine)

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "7gi6wEsghEgO"
FATAL:  password authentication failed for user "7gi6wEsghEgO"


In [38]:
c = np.max(datacounts['count'])
duse = np.array(datacounts.loc[datacounts['count'] >= c,'dataid'])

In [39]:
query = "SELECT * from university.metadata"
metadata0 = pd.read_sql_query(query, engine)
metadata = metadata0[['dataid','city']].merge(datacounts)
metadata.head()
use = (datacounts['count'] >= c ) & (metadata['city'] == 'Austin')
duse = np.array(metadata.loc[use ,'dataid'])

## Daylight Savings Time
It appears that pecan street consistently drops one hour each year when daylight savings time ends. At the onset of DST, the local time skips from 1:45 to 3:00 as it should. At the conclusion of DST, there are no additional 2am readings. Possibly this is an artifact of only the 15-minute data. 

In [40]:
## Get data from all homes with complete data (about 300)
tstart = dt.datetime.now()
query="""SELECT dataid, local_15min, use, gen, air1, air2, air3, airwindowunit1, furnace1, furnace2 
FROM university.electricity_egauge_15min
WHERE local_15min
BETWEEN '01-01-2015' AND '01-01-2016' AND electricity_egauge_15min.dataid in (""" + \
','.join([str(d) for d in duse]) + \
""");"""

the_frame = pd.read_sql_query(query,engine)
the_frame.sort_values('local_15min')
the_frame.tail()
print(dt.datetime.now() - tstart)

0:08:34.533529


In [41]:
## Localize the time and add a date column. 
the_frame["time"] = the_frame.set_index("local_15min").index.tz_localize(pytz.timezone('America/Chicago'), ambiguous = True)
the_frame["date"] = [ dt.datetime(d.year,d.month,d.day,0,0,0,0) for d in the_frame['time'] ]


In [42]:
pytz.timezone('America/Chicago')

<DstTzInfo 'America/Chicago' LMT-1 day, 18:09:00 STD>

In [43]:
loaddata = the_frame
loaddata["time"] = loaddata.set_index("local_15min").index.tz_localize(pytz.timezone('America/Chicago'), ambiguous = True)
loaddata["date"] = [ dt.datetime(d.year,d.month,d.day,0,0,0,0) for d in loaddata['time'] ]

## Dump as a pickle file. 
import pickle as pk
pk.dump( loaddata, open( "data/demand.pkl", "wb" ) )

## Weather Data
Weather data are from three locations: Austin, Boulder and San Diego.  There is no way to tell which home is in each location.  I will label and keep all of them.

In [44]:
locs =pd.read_sql_query("""SELECT distinct(latitude,longitude), latitude FROM university.weather LIMIT 10;""",engine)
locs['Location'] = ['Austin','San Diego','Boulder']
locs

,row,latitude,Location
0,"(30.292432,-97.699662)",30.292432,Austin
1,"(32.778033,-117.151885)",32.778033,San Diego
2,"(40.027278,-105.256111)",40.027278,Boulder


In [45]:
weather = pd.read_sql_query("""
SELECT * FROM university.weather  
WHERE localhour
BETWEEN '01-01-2015' AND '01-01-2016'
;
""", engine)

In [46]:
weather = weather.merge(locs[['latitude','Location']])
weather.head()

,localhour,latitude,longitude,tz_offset,summary,ozone,ozone_error,temperature,temperature_error,dew_point,...,wind_speed,wind_speed_error,cloud_cover,cloud_cover_error,wind_bearing,precip_intensity,precip_intensity_error,precip_probability,precip_type,Location
0,2015-08-23 13:00:00-05:00,40.027278,-105.256111,-6,Clear,287.11,None,67.27,None,41.72,...,2.15,None,0.13,None,66,0.0,None,0.0,None,Boulder
1,2015-09-06 03:00:00-05:00,40.027278,-105.256111,-6,Clear,276.72,None,65.97,None,43.65,...,2.05,None,0.07,None,269,0.0,None,0.0,None,Boulder
2,2015-09-25 22:00:00-05:00,40.027278,-105.256111,-6,Clear,263.95,None,63.46,None,41.04,...,0.89,None,0.00,None,160,0.0,None,0.0,None,Boulder
3,2015-09-25 23:00:00-05:00,40.027278,-105.256111,-6,Clear,262.46,None,61.34,None,41.05,...,0.68,None,0.00,None,220,0.0,None,0.0,None,Boulder
4,2015-09-26 10:00:00-05:00,40.027278,-105.256111,-6,Clear,248.03,None,60.14,None,39.87,...,0.32,None,0.00,None,43,0.0,None,0.0,None,Boulder


In [47]:
pk.dump( weather, open( "data/weather.pkl", "wb" ) )